In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
from IPython.display import HTML
from matplotlib import animation
from ipywidgets import interact, widgets
from clawpack import pyclaw
weno = pyclaw.limiters.recon.weno

In [ ]:
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [ ]:
a = np.array([1, 2, 3, 4, 5, 6]); b=rolling_window(a, 3)

# Slightly incorrect implementation
This version is correct only for forward Euler.  For other RK methods, it uses the previous step value in some places where certain combinations of stage values should instead be used.

In [ ]:
def test_advection(T=1,limit=True,order=3,nu=0.9,RKM='SSP33',IC='smooth'):

    v = 1

    def f(q):
        return v*q

    nghost = 2
    m = 50      # number of cells
    dx = 1./m   # Size of 1 grid cell
    x = np.linspace(-(2*nghost-1)*dx/2,1.+(2*nghost-1)*dx/2,m+2*nghost)
    t = 0.      # Initial time
    dt = nu * dx  # Time step

    if IC=='smooth':
        Q = np.exp(-200*(x-0.2)**2)    # Initial data
    elif IC=='square':
        Q = 1.*(x<0.2)*(x>0.1)
    H_iph = np.zeros_like(x)
    ql = np.zeros_like(x)
    qr = np.zeros_like(x)
    ubar_L_iph = np.zeros_like(x)
    H_L_iph = np.zeros_like(x)
    H_iph = np.zeros_like(x)
    f_iph = np.zeros_like(x)
    fstar_iph = np.zeros_like(x)
    umax = np.zeros_like(x)
    umin = np.zeros_like(x)

    rkm = rk.loadRKM(RKM)
    rkm = rkm.__num__()

    t = 0. # current time
    QQ = [Q.copy()] # values at each time step
    tt = np.zeros(1) # time points for ww
    tt[0] = t
    b = rkm.b
    s = len(rkm)
    y = np.zeros((s, np.size(Q))) # stage values
    G = np.zeros((s, np.size(Q))) # stage derivatives

    def donor(q):
        return q, q

    def pw_poly_recon(q,order=3):
        if order==1:
            ql[1:-1] = q[1:-1]
            qr[1:-1] = q[1:-1]
        if order==3:
            ql[1:-1] = (2.*q[:-2] + 5.*q[1:-1] - q[2:])/6.
            qr[1:-1] = (-q[:-2] + 5.*q[1:-1] + 2.*q[2:])/6.
        elif order==5:
            ql[2:-2] = (-5.*q[:-4] + 60.*q[1:-3] + 90.*q[2:-2] - 20.*q[3:-1] + 3.*q[4:])/128.
            qr[2:-2] =  (3.*q[:-4] - 20.*q[1:-3] + 90.*q[2:-2] + 60.*q[3:-1] - 5.*q[4:])/128.
        return ql, qr

    d = np.abs(v)/2.

    while t < T and not np.isclose(t, T):
        if t + dt > T:
            nu = nu*(T-t)/dt
            dt = T - t

        for i in range(s):
            y[i,:] = Q.copy()
            for j in range(i):
                y[i,:] += rkm.A[i,j]*G[j,:]
            # Reconstruction:
            ql, qr = pw_poly_recon(y[i,:],order=order)

            ubar_L_iph[1:-1] = 0.5*(Q[2:]   + Q[1:-1]) - 0.25*(f(Q[2:])  -f(Q[1:-1]))/d
            H_L_iph[:-1] = 0.5*(f(Q[:-1])+f(Q[1:])) - d*(Q[1:]-Q[:-1])
            H_iph[:-1] = 0.5*(f(qr[:-1])+f(ql[1:])) - d*(ql[1:]-qr[:-1])
            f_iph = H_L_iph - H_iph

            umax[1:-1] = np.maximum(Q[:-2],np.maximum(Q[1:-1],Q[2:]))
            umin[1:-1] = np.minimum(Q[:-2],np.minimum(Q[1:-1],Q[2:]))
            fstar_iph[:-1] = (f_iph[:-1]>0)*np.minimum(f_iph[:-1], 
                                                       2*d*np.minimum(umax[:-1]-ubar_L_iph[:-1],ubar_L_iph[:-1]-umin[1:]))\
                            +(f_iph[:-1]<0)*np.maximum(f_iph[:-1],
                                                       2*d*np.maximum(umin[:-1]-ubar_L_iph[:-1],ubar_L_iph[:-1]-umax[1:]))
        
            # Periodic boundaries:
            H_L_iph[:nghost] = H_L_iph[-2*nghost:-nghost]
            H_L_iph[-nghost:] = H_L_iph[nghost:2*nghost]
            fstar_iph[:nghost] = fstar_iph[-2*nghost:-nghost]
            fstar_iph[-nghost:] = fstar_iph[nghost:2*nghost]
            # Next two lines only needed for testing
            f_iph[:nghost] = f_iph[-2*nghost:-nghost]
            f_iph[-nghost:] = f_iph[nghost:2*nghost]
            if limit:
                G[i,1:-1] = - nu*(H_L_iph[1:-1]-H_L_iph[:-2]) + nu*(fstar_iph[1:-1]-fstar_iph[:-2])
            else:
                G[i,1:-1] = - nu*(H_L_iph[1:-1]-H_L_iph[:-2]) + nu*(f_iph[1:-1]-f_iph[:-2])

        Q = Q + sum([b[i]*G[i,:] for i in range(s)])
        Q[:nghost] = Q[-2*nghost:-nghost]
        Q[-nghost:] = Q[nghost:2*nghost]
        t += dt
        tt = np.append(tt, t)
        QQ.append(Q.copy())

    plt.plot(x[nghost:-nghost],Q[nghost:-nghost],linewidth = 2)
    plt.title('t = '+str(t));
    plt.plot(x[nghost:-nghost],QQ[0][nghost:-nghost]);
    print(np.min(Q[nghost:-nghost]))
    print(np.argmin(Q[nghost:-nghost]))

In [ ]:
interact(test_advection,T=widgets.IntSlider(min=0,max=10,step=1,value=1),
            order=widgets.IntSlider(min=1,max=5,step=2),
            nu=widgets.FloatSlider(min=0.1,max=1.,step=0.1,value=0.5),
            RKM=widgets.Dropdown(options=['FE', 'SSP33', 'SSP104','BS5'],value='FE',description='RK method:'),
            IC=widgets.Dropdown(options=['smooth', 'square'],value='smooth',description='Initial cond.:'),
        
        );

# Shu-Osher implementation

In [ ]:
def test_advection(T=1,convex_limiting='local',order=3,nu=0.9,RKM='SSP33',m=100,weno_limiting=True):

    v = 1

    def f(q):
        return v*q

    nghost = 2
    dx = 1./m   # Size of 1 grid cell
    x = np.linspace(-(2*nghost-1)*dx/2,1.+(2*nghost-1)*dx/2,m+2*nghost)
    t = 0.      # Initial time
    dt = nu * dx  # Time step
        
    beta=200.; x0=0.3
    Q = np.exp(-beta * (x-x0)**2) + (x>0.6)*(x<0.8)
    
    H_iph = np.zeros_like(x)
    ql = np.zeros_like(x)
    qr = np.zeros_like(x)
    ubar_L_iph = np.zeros_like(x)
    H_L_iph = np.zeros_like(x)
    H_iph = np.zeros_like(x)
    f_iph = np.zeros_like(x)
    fstar_iph = np.zeros_like(x)
    umax = np.zeros_like(x)
    umin = np.zeros_like(x)

    rkm = rk.loadRKM(RKM)
    rkm = rkm.__num__()

    t = 0. # current time
    QQ = [Q.copy()] # values at each time step
    tt = np.zeros(1) # time points for ww
    tt[0] = t
    b = rkm.b
    s = len(rkm)
    y = np.zeros((s, np.size(Q))) # stage values
    G = np.zeros((s, np.size(Q))) # stage derivatives

    def donor(q):
        return q, q

    def pw_poly_recon(q,order=3,weno_limiting=True):
        if weno_limiting:
            assert(order == 5)
            ql[:], qr[:] = weno(5,q.reshape(1,len(q)))
        elif order==1:
            ql[1:-1] = q[1:-1]
            qr[1:-1] = q[1:-1]
        elif order==3:
            ql[1:-1] = (2.*q[:-2] + 5.*q[1:-1] - q[2:])/6.
            qr[1:-1] = (-q[:-2] + 5.*q[1:-1] + 2.*q[2:])/6.
        elif order==5:
            ql[2:-2] = (-5.*q[:-4] + 60.*q[1:-3] + 90.*q[2:-2] - 20.*q[3:-1] + 3.*q[4:])/128.
            qr[2:-2] =  (3.*q[:-4] - 20.*q[1:-3] + 90.*q[2:-2] + 60.*q[3:-1] - 5.*q[4:])/128.
        return ql.squeeze(), qr.squeeze()

    def euler_step(Q, nu, dt_factor):
        u = Q.copy()
        ql, qr = pw_poly_recon(u,order=order,weno_limiting=weno_limiting)

        ubar_L_iph[1:-1] = 0.5*(u[2:]   + u[1:-1]) - 0.25*(f(u[2:])  -f(u[1:-1]))/d
        H_L_iph[:-1] = 0.5*(f(u[:-1])+f(u[1:])) - d*(u[1:]-u[:-1])
        H_iph[:-1] = 0.5*(f(qr[:-1])+f(ql[1:])) - d*(ql[1:]-qr[:-1])
        f_iph = H_L_iph - H_iph

        if convex_limiting == 'local':            
            umax[nghost:-nghost] = np.max(rolling_window(u,2*nghost+1),axis=1)
            umin[nghost:-nghost] = np.min(rolling_window(u,2*nghost+1),axis=1)
        elif convex_limiting == 'global':
            umax[:] = np.ones_like(u)
            umin[:] = np.zeros_like(u)
            
        fstar_iph[:-1] = (f_iph[:-1]>0)*np.minimum(f_iph[:-1], 
                                                   2*d*np.minimum(umax[:-1]-ubar_L_iph[:-1],ubar_L_iph[:-1]-umin[1:]))\
                        +(f_iph[:-1]<0)*np.maximum(f_iph[:-1],
                                                   2*d*np.maximum(umin[:-1]-ubar_L_iph[:-1],ubar_L_iph[:-1]-umax[1:]))

        # Periodic boundaries:
        H_L_iph[:nghost] = H_L_iph[-2*nghost:-nghost]
        H_L_iph[-nghost:] = H_L_iph[nghost:2*nghost]
        fstar_iph[:nghost] = fstar_iph[-2*nghost:-nghost]
        fstar_iph[-nghost:] = fstar_iph[nghost:2*nghost]
        # Next two lines only needed for testing
        f_iph[:nghost] = f_iph[-2*nghost:-nghost]
        f_iph[-nghost:] = f_iph[nghost:2*nghost]
        if convex_limiting in ['local','global']:
             u[1:-1] = u[1:-1] - dt_factor*nu*(H_L_iph[1:-1]-H_L_iph[:-2] - fstar_iph[1:-1] + fstar_iph[:-2])
        else:
             u[1:-1] = u[1:-1] - dt_factor*nu*(H_L_iph[1:-1]-H_L_iph[:-2] - f_iph[1:-1] + f_iph[:-2]) 
        return u
        
    d = np.abs(v)/2.

    while t < T and not np.isclose(t, T):
        if t + dt > T:
            nu = nu*(T-t)/dt
            dt = T - t

        if RKM=='FE':
            Q = euler_step(Q,nu,1.0)
        elif RKM == 'SSP22':
            y2 = euler_step(Q,nu,1.0)
            y2[:nghost] = y2[-2*nghost:-nghost]
            y2[-nghost:] = y2[nghost:2*nghost]
            Q = 0.5*Q + 0.5*euler_step(y2,nu,1.0)
        elif RKM == 'SSP33':
            y2 = euler_step(Q,nu,1.0)
            y2[:nghost] = y2[-2*nghost:-nghost]
            y2[-nghost:] = y2[nghost:2*nghost]
            y3 = 0.75*Q + 0.25*euler_step(y2,nu,1.0)
            y3[:nghost] = y3[-2*nghost:-nghost]
            y3[-nghost:] = y3[nghost:2*nghost]            
            Q = 1./3*Q + 2./3*euler_step(y3,nu,1.0)
            
        Q[:nghost] = Q[-2*nghost:-nghost]
        Q[-nghost:] = Q[nghost:2*nghost]
        t += dt
        tt = np.append(tt, t)
        QQ.append(Q.copy())

    plt.plot(x[nghost:-nghost],Q[nghost:-nghost],linewidth = 2)
    plt.title('t = '+str(t));
    plt.plot(x[nghost:-nghost],QQ[0][nghost:-nghost],'--k',alpha=0.5);
    print(np.min(Q[nghost:-nghost]))

In [ ]:
interact(test_advection,T=widgets.IntSlider(min=0,max=10,step=1,value=1),
            order=widgets.IntSlider(min=1,max=5,step=2,value=5),
            nu=widgets.FloatSlider(min=0.1,max=1.,step=0.1,value=0.5,description='$\\nu$'),
            RKM=widgets.Dropdown(options=['FE', 'SSP22', 'SSP33'],value='SSP33',description='RK method:'),
            m=widgets.IntText(value=100),
            convex_limiting=widgets.Dropdown(options=['local','global','none']),
        );

# Convex limiting in time

In [ ]:
Min = np.minimum
Max = np.maximum

def test_advection(T=1,convex_limiting=True,limit_time=False,order=5,nu=0.9,RKM='SSP33',m=100,weno_limiting=True):

    v = 1

    def f(q):
        return v*q

    nghost = 2
    dx = 1./m   # Size of 1 grid cell
    x = np.linspace(-(2*nghost-1)*dx/2,1.+(2*nghost-1)*dx/2,m+2*nghost)
    t = 0.      # Initial time
    dt = nu * dx  # Time step

    beta=200.; x0=0.3
    Q = np.exp(-beta * (x-x0)**2) + (x>0.6)*(x<0.8)
    
    H_iph = np.zeros_like(x)
    ql = np.zeros_like(x)
    qr = np.zeros_like(x)
    ubar_L_iph = np.zeros_like(x)
    H_L_iph = np.zeros_like(x)
    H_iph = np.zeros_like(x)
    u_iph = np.zeros_like(x)
    f_iph = np.zeros_like(x)
    fstar_iph = np.zeros_like(x)
    umax = np.zeros_like(x)
    umin = np.zeros_like(x)

    rkm = rk.loadRKM(RKM)
    rkm = rkm.__num__()

    t = 0. # current time
    QQ = [Q.copy()] # values at each time step
    tt = np.zeros(1) # time points for ww
    tt[0] = t
    b = rkm.b
    s = len(rkm)
    y = np.zeros((s, np.size(Q))) # stage values
    G = np.zeros((s, np.size(Q))) # stage derivatives

    def apply_bcs(q):
        q[:nghost] = q[-2*nghost:-nghost]
        q[-nghost:] = q[nghost:2*nghost]
        
    def donor(q):
        return q, q

    def pw_poly_recon(q,order=3,weno_limiting=True):
        if weno_limiting:
            assert(order == 5)
            ql[:], qr[:] = weno(5,q.reshape(1,len(q)))
        elif order==1:
            ql[1:-1] = q[1:-1]
            qr[1:-1] = q[1:-1]
        elif order==3:
            ql[1:-1] = (2.*q[:-2] + 5.*q[1:-1] - q[2:])/6.
            qr[1:-1] = (-q[:-2] + 5.*q[1:-1] + 2.*q[2:])/6.
        elif order==5:
            ql[2:-2] = (-5.*q[:-4] + 60.*q[1:-3] + 90.*q[2:-2] - 20.*q[3:-1] + 3.*q[4:])/128.
            qr[2:-2] =  (3.*q[:-4] - 20.*q[1:-3] + 90.*q[2:-2] + 60.*q[3:-1] - 5.*q[4:])/128.
        return ql.squeeze(), qr.squeeze()

    def euler_step(Q, nu, dt_factor):
        fstar_iph = np.zeros_like(x)
        u = Q.copy()
        ul, ur = pw_poly_recon(u,order=order,weno_limiting=weno_limiting)

        # For advection, these are just the upwind states:
        u_iph[1:-1] = 0.5*(u[2:]   + u[1:-1]) - 0.5*(f(u[2:]) - f(u[1:-1]))/v # middle Riemann state from LLF
        H_L_iph[:-1] = 0.5*(f(u[:-1])+f(u[1:])) - 0.5*v*(u[1:]-u[:-1])  # LLF flux with low-order inputs
        H_iph[:-1] = 0.5*(f(ur[:-1])+f(ul[1:])) - 0.5*v*(ul[1:]-ur[:-1]) # LLF flux with high-order inputs
        f_iph = H_iph - H_L_iph  # "correction" flux (with minus sign)
        
        #test = H_L_iph[:-2]-H_L_iph[1:-1] - ((ubar_L_iph[1:-1]-Q[1:-1])+(ubar_L_iph[:-2]-Q[1:-1]))
        #print(t,' ',np.max(np.abs(test[nghost:-nghost])))  # Checks out

        if convex_limiting == 'local':            
            umax[nghost:-nghost] = np.max(rolling_window(u,2*nghost+1),axis=1)
            umin[nghost:-nghost] = np.min(rolling_window(u,2*nghost+1),axis=1)
        elif convex_limiting == 'global':
            umax[:] = np.ones_like(u)
            umin[:] = np.zeros_like(u)
        
        umax[1:-1] = Max(u[:-2],Max(u[1:-1],u[2:]))
        umin[1:-1] = Min(u[:-2],Min(u[1:-1],u[2:]))
        f_iph = -f_iph
        fstar_iph[:-1] = (f_iph[:-1]>0)*Min(f_iph[:-1], 
                                                   v*Min(umax[:-1]-u_iph[:-1],u_iph[:-1]-umin[1:]))\
                        +(f_iph[:-1]<0)*Max(f_iph[:-1],
                                                   v*Max(umin[:-1]-u_iph[:-1],u_iph[:-1]-umax[1:]))
        
        f_iph = -f_iph
        fstar_iph = -fstar_iph

        # Periodic boundaries:
        apply_bcs(H_L_iph)
        apply_bcs(fstar_iph)
        apply_bcs(f_iph)

        ##########
        # Check some stuff
        #uH_iph = u_iph + f_iph/v
        #print(ur-uH_iph)
        ###########
        
        if convex_limiting in ['local','global']:
            u[1:-1] = u[1:-1] - dt_factor*nu*(H_L_iph[1:-1]-H_L_iph[:-2] + fstar_iph[1:-1] - fstar_iph[:-2])
        else:
             u[1:-1] = u[1:-1] - dt_factor*nu*(H_L_iph[1:-1]-H_L_iph[:-2] + f_iph[1:-1] - f_iph[:-2])
        ubarstar_iph = u_iph + fstar_iph/v
        
        #test1 = -H_L_iph[1:-1] + H_L_iph[:-2] - fstar_iph[1:-1] + fstar_iph[:-2]
        #test2 = -(ubarstar_iph[1:-1] - u[1:-1]) + (ubarstar_iph[:-2]-u[1:-1])
        #print(np.max(np.abs(test1[2:-2]-test2[2:-2])))

        return u, H_L_iph-fstar_iph, ubarstar_iph
        

    while t < T and not np.isclose(t, T):
        if t + dt > T:
            nu = nu*(T-t)/dt
            dt = T - t

        if RKM=='FE':
            # This is really hacky!
            if limit:
                _, H1, ubarstar_iph = euler_step(Q,nu,1.0)
                Q[1:-1] = Q[1:-1] + v*nu*(-(ubarstar_iph[1:-1]-Q[1:-1])+(ubarstar_iph[:-2]-Q[1:-1])) # Doesn't check out!
            else:
                Q, H1, ubarstar_iph = euler_step(Q,nu,1.0)
                
        elif RKM == 'SSP22':
            y2, H1, ubarstar_iph = euler_step(Q,nu,1.0)
            apply_bcs(y2)
            y3, H2, _ = euler_step(y2,nu,1.0)
            if limit_time:
                fdot = H1 - 0.5*(H1+H2)
                umax[1:-1] = Max(Q[:-2],Max(Q[1:-1],Q[2:]))
                umin[1:-1] = Min(Q[:-2],Min(Q[1:-1],Q[2:]))
                fdotstar_iph = (fdot>0)*(Min(fdot,v*Min(umax-ubarstar_iph,ubarstar_iph-umin))) \
                          +(fdot<0)*(Max(fdot,v*Max(umin-ubarstar_iph,ubarstar_iph-umax)))  # Eqn. (32)
                #Q = Q + nu*(v*(ubarstar_iph-Q)+fdotstar_iph)
                #Q[1:-1] = Q[1:-1] + nu*(v*(-(ubarstar_iph[1:-1]-Q[1:-1]) + ubarstar_iph[:-2]-Q[1:-1])+fdot[1:-1]-fdot[:-2])
                Q[1:-1] = y2[1:-1] + nu*(fdot[1:-1]-fdot[:-2])  # good
            else:
                Q = 0.5*Q + 0.5*y3
        elif RKM == 'SSP33':
            y2, H1, ubarstar_iph = euler_step(Q,nu,1.0)
            apply_bcs(y2)
            y3 = 0.75*Q + 0.25*euler_step(y2,nu,1.0)
            apply_bcs(y3)     
            Q = 1./3*Q + 2./3*euler_step(y3,nu,1.0)
            
        apply_bcs(Q)
        t += dt
        tt = np.append(tt, t)
        QQ.append(Q.copy())

    plt.plot(x[nghost:-nghost],Q[nghost:-nghost],linewidth = 2)
    plt.title('t = '+str(t));
    plt.plot(x[nghost:-nghost],QQ[0][nghost:-nghost],'--k',alpha=0.5);
    print(np.min(Q[nghost:-nghost]))
    print(np.argmin(Q[nghost:-nghost]))

In [ ]:
interact(test_advection,T=widgets.IntSlider(min=0,max=10,step=1,value=1),
            order=widgets.IntSlider(min=1,max=5,step=2,value=5),
            nu=widgets.FloatSlider(min=0.1,max=1.,step=0.1,value=0.5,description='$\\nu$'),
            RKM=widgets.Dropdown(options=['FE', 'SSP22', 'SSP33'],value='SSP22',description='RK method:'),
            m=widgets.IntText(value=100),
            convex_limiting=widgets.Dropdown(options=['local','global','none']),
        );